In [116]:
!pip install nltk

In [1]:
import pandas as pd
import json
from nltk.corpus import brown
from urllib.parse import urlparse
import nltk
import re
import os
PATH=os.getcwd()
#nltk.download('brown')
import string
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
def get_website(x):
    x = urlparse(x).netloc
    return x
def filter_tags(taglist, history):
    '''
    Creates a list (tags_in) of the words in the original list that are also the full_data column.
    drops words from list that are also in corpus. drops words with len < 
    '''
    tags_in = []

    for tag in taglist:
        count = history["clean_full_data"].str.contains(tag, regex=False).sum()
        if (count>0) :
            tags_in.append(tag)

    return tags_in
## Utils

# get decrption 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import xml.etree.ElementTree as et

from urllib.parse import urlparse

url = 'https://reactjs.org/'
response = requests.get(url)
soup = BeautifulSoup(response.text)

metas = soup.find_all('meta')

ls= [ meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description' ]

def get_q(x):
    x = urlparse(x).query
    return x
def get_fragment(x):
    x = urlparse(x).fragment
    return x
import pandas as pd
import numpy as np
import re
import xml.etree.ElementTree as et

from urllib.parse import urlparse

CUT_OFF= 1800

def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    
    
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

def xml_to_list (xml_file):
    '''
    takes file location 'xml_file' and gives a list of the values in the xml_file.
    '''
    df=parse_XML(xml_file, ["TagName"])
    df['TagName']=" "+df['TagName']+" "
    df['TagName'].replace("-"," ",inplace=True,regex=True)
    tag_ls=(df['TagName']).tolist()
    tags_to_drop = [' facebook ', ' twitter ', ' gmail ',
            ' google forms ', ' profile ', ' inbox ', 
            ' email ', ' drive ', ' spotify ', 
            ' search ', ' whatsapp ', ' join ', 
            ' view ', ' agenda ', ' google docs ', 
            ' free ', ' add ', ' release ', 
            ' seconds ', ' linkedin ', 'xiaomi'
            ' global ', ' launch ', 
            ' post ', ' using ', ' this ']

    for tag in tag_ls:
        if (len(tag) < 5) or tag in tags_to_drop:
            tag_ls.remove(tag)
    

    return tag_ls

def get_website(x):
    x = urlparse(x).netloc
    return x

def create_history(csv_file):
    '''
    reads csv file into DF 'history', and:
    1. parse dates
    3. creates 'website' column
    2. creates 'full_data' column - title + url in lowercase
    3. removes special charecters from full_data
    
    csv_file should be entered with r'file.csv' (r and '')

    '''
    history = pd.read_csv(csv_file, parse_dates=[['date', 'time']])
    history['website'] = history.url.apply(get_website)
    history['full_data'] = (history['url']+" "+history['title']+" ").str.lower()
    history['full_data'] = (history['full_data']).str.replace(r'[^\w\s]+', ' ', regex=True)
        
    return history

def time_spent (x):
    '''
    creates column with time spent in website session (in seconds)
    if the time spent is over 30 min (1800 seconds = cut off), the time spent column will display the median for the website
    '''
    #time spent in website (in seconds)
    history['end_time'] = history.date_time.shift(1)
    history['time_spent'] = (history.end_time - history.date_time).dt.seconds
    #create Nan values:
    history['time_spent'] = history.time_spent.apply(lambda x: np.nan if x >= CUT_OFF else x)
    #NaN values to website median
    grouped = history.groupby(['website']).time_spent
    history['time_spent'] = grouped.transform(lambda x: x.fillna(x.median()))
    history.drop('end_time', axis='columns')

def filter_tags(taglist, corpus, history):
    '''
    Creates a list (tags_in) of the words in the original list that are also the full_data column.
    drops words from list that are also in corpus. drops words with len < 
    '''
    tags_in = []
    word_list = corpus.words()

    for tag in tag_ls:
        count = history["clean_full_data"].str.contains(tag, regex=False).sum()
        if (count>0) and (tag.replace(' ','') not in word_list):
            tags_in.append(tag)

    return tags_in

def is_study (tags, urls):
    '''
    returns a list 'match_tags' of tags that appear in each row in history, by order.
    if no tag is found, the value is set to 'irrelevent'.
    '''    
    match_tags = ["irrelevent"] * len(urls)
    
    for tag in tags:
        # compile regex per tag (here so we dont do this again in the nested loop)
        match_string = re.compile(f".*{tag}.*")
        for i,url in enumerate(urls):
            if re.fullmatch(match_string, str(url)):
                if match_tags[i] == "irrelevent":
                    match_tags[i] = tag
                else:
                    match_tags[i] += f", {tag}"        
        history[tag] = history['full_data'].str.contains(tag, regex=False)

    return match_tags


In [6]:
import pandas as pd
import json
from nltk.corpus import brown
from urllib.parse import urlparse
import nltk
import re
import os
PATH=os.getcwd()

# Opening JSON file and read file
f = open(os.path.join(PATH,r"BrowserHistory.json"))
data = json.load(f)
f.close()
history=pd.json_normalize(data['Browser History'])

tags=pd.read_excel(os.path.join(PATH,r"tags.xlsx"))
word_list = brown.words()
tags['rank']=tags.index
# returns JSON object as
# a dictionary


history['time']=pd.to_datetime(history['time_usec'], unit='us')
history['website']=history['url'].apply(get_website)
history["new_url"]= history.apply(lambda x: x['url'].replace(r"https://"+x['website']+r"/", ""), axis=1)
history['full_data'] = (history['new_url']+" "+history['title']+" ").str.lower()


In [12]:
history['clean_text'] = history['full_data'].apply(lambda x: preprocess(x))

In [20]:
history.groupby("website").count().sort_values("clean_text").tail(20)

,favicon_url,page_transition,title,url,client_id,time_usec,time,new_url,full_data,clean_text
website,,,,,,,,,,
m.facebook.com,4,167,167,167,167,167,167,167,167,167
discord.com,28,172,172,172,172,172,172,172,172,172
webhook.site,159,193,193,193,193,193,193,193,193,193
analytics.google.com,114,206,206,206,206,206,206,206,206,206
www.linkedin.com,144,208,208,208,208,208,208,208,208,208
localhost:8888,213,214,214,214,214,214,214,214,214,214
m.imdb.com,0,219,219,219,219,219,219,219,219,219
www.tase.co.il,217,227,227,227,227,227,227,227,227,227
www.bizportal.co.il,212,231,231,231,231,231,231,231,231,231


In [5]:
## tags analysis
new_tags_ls=set(tags['Attribute:TagName'].tolist()) - set(word_list)
dictionary_tags_ls=set(tags['Attribute:TagName'].tolist()) & set(word_list)
# bring back top 100
top_100=tags[tags['Attribute:TagName'].isin(dictionary_tags_ls)].loc[:100]
new_tags=tags[tags['Attribute:TagName'].isin(new_tags_ls)]
new_tags=pd.concat([new_tags,top_100])
new_tags=new_tags.sort_values("rank")
# clean tags
new_tags['clean_tags']=new_tags['Attribute:TagName'].replace("-"," ",regex=True)
history['full_data'] = (history['new_url']+" "+history['title']+" ").str.lower()
# delete non-aplha numeric
history['clean_full_data']=history['full_data'].str.replace(r'[^A-Za-z0-9 ]+', ' ')
# delete multiple spaces
history['clean_full_data'] = history['clean_full_data'].replace('\s+', ' ', regex=True)

,favicon_url,page_transition,title,url,client_id,time_usec,time,website,new_url,full_data,clean_full_data
0,https://www.thepythoncode.com/static/img/pytho...,LINK,How to Read Emails in Python - Python Code,https://www.thepythoncode.com/article/reading-...,ChCEMzICD0Hcp6qr+O44Tw==,1636729058732261,2021-11-12 14:57:38.732261,www.thepythoncode.com,article/reading-emails-in-python,article/reading-emails-in-python how to read e...,article reading emails in python how to read e...
1,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/settings/takeout?pli=1,ChCEMzICD0Hcp6qr+O44Tw==,1636729053479999,2021-11-12 14:57:33.479999,takeout.google.com,settings/takeout?pli=1,settings/takeout?pli=1 google takeout,settings takeout pli 1 google takeout
2,NaN,LINK,Google Takeout,https://takeout.google.com/settings/takeout?pli=1,ChCEMzICD0Hcp6qr+O44Tw==,1636729051496083,2021-11-12 14:57:31.496083,takeout.google.com,settings/takeout?pli=1,settings/takeout?pli=1 google takeout,settings takeout pli 1 google takeout
3,https://www.google.com/favicon.ico,GENERATED,google takeout - חיפוש ב-Google,https://www.google.com/search?q=google+takeout...,ChCEMzICD0Hcp6qr+O44Tw==,1636729048837442,2021-11-12 14:57:28.837442,www.google.com,search?q=google+takeout&oq=google+takeout&aqs=...,search?q=google+takeout&oq=google+takeout&aqs=...,search q google takeout oq google takeout aqs ...
4,https://www.google.com/favicon.ico,GENERATED,python get latest emails in inbox - חיפוש ב-Go...,https://www.google.com/search?q=python+get+lat...,ChCEMzICD0Hcp6qr+O44Tw==,1636729042414008,2021-11-12 14:57:22.414008,www.google.com,search?q=python+get+latest+mails+in+inbox&oq=p...,search?q=python+get+latest+mails+in+inbox&oq=p...,search q python get latest mails in inbox oq p...
...,...,...,...,...,...,...,...,...,...,...,...
20950,NaN,LINK,LinkedIn Profile Scraper | Phantombuster,https://phantombuster.com/automations/linkedin...,dRUjB3pqPv09pCdNCc+N5A==,1605253150283348,2020-11-13 07:39:10.283348,phantombuster.com,automations/linkedin/3112/linkedin-profile-scr...,automations/linkedin/3112/linkedin-profile-scr...,automations linkedin 3112 linkedin profile scr...
20951,NaN,RELOAD,Registration page,https://trailer.web-view.net/LandingPagesResul...,dRUjB3pqPv09pCdNCc+N5A==,1605252110781953,2020-11-13 07:21:50.781953,trailer.web-view.net,LandingPagesResult.aspx?_p=f3f2a2&_u=333waaac&...,landingpagesresult.aspx?_p=f3f2a2&_u=333waaac&...,landingpagesresult aspx p f3f2a2 u 333waaac c ...
20952,NaN,LINK,מהדורה-דיגיטלית | כל העיר ירושלים,https://www.kolhair.co.il/digital-edition/,dRUjB3pqPv09pCdNCc+N5A==,1605208329563089,2020-11-12 19:12:09.563089,www.kolhair.co.il,digital-edition/,digital-edition/ מהדורה-דיגיטלית | כל העיר ירו...,digital edition
20953,NaN,LINK,מהדורה-דיגיטלית | כל העיר ירושלים,https://www.kolhair.co.il/digital-edition/#,dRUjB3pqPv09pCdNCc+N5A==,1605208321008995,2020-11-12 19:12:01.008995,www.kolhair.co.il,digital-edition/#,digital-edition/# מהדורה-דיגיטלית | כל העיר יר...,digital edition


In [72]:
# take few Mintues
ls_tag_in=filter_tags((r" "+(new_tags['clean_tags'])+r" ").tolist(),history)

In [94]:
ser_tags_in=pd.Series(ls_tag_in)
ser_tags_in=ser_tags_in[ser_tags_in.str.len()>5]

In [95]:
ser_tags_in.drop_duplicates().to_excel(r"C:\Users\meira\tovtech\tags_in.xlsx")

In [ ]:

# genrete report
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import plotly.express as px
import cufflinks as cf
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
time_df = pd.DataFrame(columns=["tag","time"])

tag_list = history.columns.tolist()[13:-2]
time_df['tag'] = tag_list
time_list = []

for tag in tag_list:
    tag_time = (history.loc[history[tag] == 1, 'time_spent'].sum())
    time_list.append(tag_time)

time_df['time'] = time_list
time_df.sort_values('time', ascending=False, inplace=True)
time_df.reset_index(drop=True, inplace=True)
time_df.loc[10:, "tag"] = "Other topic"
fig = px.pie(time_df, values='time', names='tag', title='Time spent on topic')
fig.show()